In [2]:
from datasets import load_dataset

ds = load_dataset("GonzaloA/fake_news")

C:\Users\guyor\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split:   0%|          | 0/24353 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8117 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8117 [00:00<?, ? examples/s]

KeyError: "Invalid key: slice(None, 150, None). Please first select a split. For example: `my_dataset_dictionary['train'][slice(None, 150, None)]`. Available splits: ['test', 'train', 'validation']"

In [224]:
ds['train'][0]

{'Unnamed: 0': 0,
 'title': ' ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)',
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the capti

In [225]:
ds = ds.rename_column('Unnamed: 0', 'index')


In [58]:
def transform_row_to_prompt(row):
    return {
        "messages": [
            {
                "role": "system",
                "content": "Given the title and text of a news article, return 0 if the article is fake news and 1 if the article is real news."
            },
            {
                "role": "user",
                "content": f"title:{row['title']} text:{row['text']}"
            },
            {
                "role": "assistant",
                "content": str(row['label'])
            }
        ]
    }

# Example usage:
transformed_row = transform_row_to_prompt(ds['train'][2])
print(transformed_row)


{'messages': [{'role': 'system', 'content': 'Given the title and text of a news article, return 0 if the article is fake news and 1 if the article is real news.'}, {'role': 'user', 'content': 'title:Russia warns Iraq, Kurds not to destabilize Middle East after Kurdish vote text:MOSCOW (Reuters) - Russia on Wednesday warned Iraq and the Kurds against taking any steps that might destabilize the Middle East after a Kurdish independence referendum, encouraging both sides to hold talks to find a solution within the framework of a single Iraqi state. The Russian Foreign Ministry, in the same statement, also said that while Moscow respected the Kurds  national ambitions it favored preserving the territorial integrity of Iraq. '}, {'role': 'assistant', 'content': '1'}]}


#### found that our data is not clean, the newspaper's name in which the article was published in appears in the text. 

## - playing with the data.

In [30]:
import re

In [ ]:
def clean_row_from_news_or_social_media_names(row):
    media_names = [
        "Reuters", "CNN", "BBC", "Al Jazeera", "Fox News", "New York Times", 
        "Washington Post", "Facebook", "Twitter", "Instagram", "YouTube"
    ]
    
    # Create a regex pattern that matches any of the media names
    media_pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, media_names)) + r')\b', re.IGNORECASE)
    
    # Replace media names with an empty string
    row['text'] = media_pattern.sub('',row['text'])
    row['title'] = media_pattern.sub('',row['title'])


In [31]:
from collections import defaultdict

# Global dictionary to aggregate counts
global_expression_counts = defaultdict(int)

def count_bracketed_expressions(text):
    global global_expression_counts
    
    # Regular expression to match all text within brackets (parentheses)
    bracket_pattern = re.compile(r'\((.*?)\)')
    
    # Find all bracketed expressions in the text
    bracketed_expressions = bracket_pattern.findall(text)
    
    # Update the global dictionary with counts
    for expression in bracketed_expressions:
        global_expression_counts[expression] += 1


In [32]:
# Iterate over the dataset and count bracketed expressions
for row in ds['train']:
    count_bracketed_expressions(row['title'])
    count_bracketed_expressions(row['text'])
    
# Print the aggregated counts
for expression, count in global_expression_counts.items():
    print(f"'{expression}': {count} time(s)")


'IMAGE': 101 time(s)
'@realDonaldTrump': 978 time(s)
'@FoxNews': 64 time(s)
'@ucla_007': 1 time(s)
'@RealPatriot1976': 1 time(s)
'@Lampliighter': 1 time(s)
'@CandidateBill': 1 time(s)
'@letmesharewithu': 1 time(s)
'@geofftype': 1 time(s)
'@keishhhha': 1 time(s)
'@Kellyk1969': 1 time(s)
'@pvliberal': 1 time(s)
'@Melanie03630436': 1 time(s)
'@DiceTravels': 1 time(s)
'@AngryHatter': 1 time(s)
'@Windemere22': 1 time(s)
' ': 21 time(s)
'@hnelke1973': 1 time(s)
'@DoreeGraham': 1 time(s)
'@erin0331': 1 time(s)
'@sarapdaily': 2 time(s)
'@adityaksharma': 1 time(s)
'@R_A_Bell': 1 time(s)
'Reuters': 12664 time(s)
'R., La.': 3 time(s)
'@SteveScalise': 5 time(s)
'VIDEO': 1468 time(s)
'CPJ': 1 time(s)
'@NYDailyNews': 6 time(s)
'D-CA': 34 time(s)
'I-VT': 16 time(s)
'at an openly scheduled W.H. meeting': 2 time(s)
'@KeithOlbermann': 7 time(s)
'the advisers': 1 time(s)
'in the West': 1 time(s)
'NORAD': 3 time(s)
'subject to Senate confirmation': 39 time(s)
'ICBM': 24 time(s)
'@JxhnBinder': 2 time(s)
'@

## - fine tuning chat gpt.

### turning the train data to prompts in a json file.

In [59]:
import json

output = []

for row in ds['train']:
    transformed_row = transform_row_to_prompt(row)
    output.append(transformed_row)


In [136]:
# Define the output file name
output_file = 'train_prompts_for_fine_tuning_gpt.jsonl'

# Write the output to a JSONL file. JSONL!!!
with open(output_file, 'w') as f:
    for item in output:
        f.write(json.dumps(item) + '\n')

print(f"Dataset has been transformed and saved to {output_file}")

Dataset has been transformed and saved to train_prompts_for_fine_tuning_gpt.jsonl


In [137]:
output_file

'train_prompts_for_fine_tuning_gpt.jsonl'

In [63]:
import openai
from openai import OpenAI

In [118]:
key = ""
client = OpenAI(api_key= key)

#### uploading output file.

In [138]:
# client.files.create(
#   file=open(output_file, "rb"),
#   purpose="fine-tune"
# )

FileObject(id='file-zscJDritTEA4roDpxChgpMVF', bytes=68871777, created_at=1724602544, filename='train_prompts_for_fine_tuning_gpt.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [135]:
output_file

'train_prompts_for_fine_tuning_gpt.json'

#### creating a fined - tuned model.

In [142]:
# job = client.fine_tuning.jobs.create(
#   training_file="file-zscJDritTEA4roDpxChgpMVF", 
#   model="gpt-4o-mini-2024-07-18"
# )

In [179]:
# job = job.to_dict()
job

{'id': 'ftjob-mhn98wtmxtGrJhldNzF9XMoi',
 'created_at': 1724602655,
 'error': {},
 'fine_tuned_model': None,
 'finished_at': None,
 'hyperparameters': {'n_epochs': 'auto',
  'batch_size': 'auto',
  'learning_rate_multiplier': 'auto'},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'fine_tuning.job',
 'organization_id': 'org-xMOF7AfWdhJun9KhQDgVieHs',
 'result_files': [],
 'seed': 1460229121,
 'status': 'validating_files',
 'trained_tokens': None,
 'training_file': 'file-zscJDritTEA4roDpxChgpMVF',
 'validation_file': None,
 'estimated_finish': None,
 'integrations': [],
 'user_provided_suffix': None}

In [182]:
job_status = client.fine_tuning.jobs.retrieve("ftjob-mhn98wtmxtGrJhldNzF9XMoi").to_dict()
job_status

{'id': 'ftjob-mhn98wtmxtGrJhldNzF9XMoi',
 'created_at': 1724602655,
 'error': {},
 'fine_tuned_model': 'ft:gpt-4o-mini-2024-07-18:personal::A0BjwIk7',
 'finished_at': 1724608491,
 'hyperparameters': {'n_epochs': 1,
  'batch_size': 16,
  'learning_rate_multiplier': 1.8},
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'fine_tuning.job',
 'organization_id': 'org-xMOF7AfWdhJun9KhQDgVieHs',
 'result_files': ['file-shSTGRCRtar6LNYXj6Rf3Ngw'],
 'seed': 1460229121,
 'status': 'succeeded',
 'trained_tokens': 13910844,
 'training_file': 'file-zscJDritTEA4roDpxChgpMVF',
 'validation_file': None,
 'estimated_finish': None,
 'integrations': [],
 'user_provided_suffix': None}

## - Claud 

### zero shot experiment.
#### generate prompts from data base

In [94]:
def Claude_transform_row_to_ZERO_shot_massage(row):
        messages = [
           {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":f"title:{row['title']} text:{row['text']}"
                    }
                ]
            }
        ]
        return messages

In [68]:
import anthropic

In [69]:
claud_client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="",
)

In [167]:
claud_client

In [160]:
# Claude_transform_row_to_ZERO_shot_massage(ds['train'][2])

In [110]:
ret_msg = claud_client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=100,
    temperature=0,
    system="Your task is to classify a given title and text of a news article as being fake news or real news."
           "return strictly a binary classification: '0' if the article is fake news, '1' if the article is real news."
           "do not return any explanations, only '0' or '1'",
    messages= Claude_transform_row_to_ZERO_shot_massage(ds['train'][23])
)

In [ ]:
import time

In [ ]:
def count_words_in_request_message(request_message):
    # Extract the system message and the transformed message from the request_message
    system_message = request_message["system"]
    transformed_message = request_message["messages"]
    
    # Concatenate the system message and the transformed message
    full_message = system_message + " " + transformed_message
    
    # Split the full message into words and count them
    word_count = len(full_message.split())
    
    return word_count

### - function for not exceeding the free use limits of claude

In [ ]:
def classify_articles(claud_client, dataset, model_name = "claude-3-opus-20240229"):
    max_requests_per_minute = 5
    max_tokens_per_minute = 10000
    max_tokens_per_day = 300000

    total_tokens_used = 0
    start_time = time.time()

    classifications = []
    
    for i in range(10):
        if i > 0 and i % max_requests_per_minute == 0:
            # Enforce the requests per minute limit
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            start_time = time.time()

        # Generate the request message
        request_message = {
            "model": model_name,
            "max_tokens": 100,
            "temperature": 0,
            "system": "Your task is to classify a given title and text of a news article as being fake news or real news."
                      "Return strictly a binary classification: '0' if the article is fake news, '1' if the article is real news."
                      "Do not return any explanations, only '0' or '1'",
            "messages": Claude_transform_row_to_ZERO_shot_massage(dataset[i])
        }

        # Send the request
        index, ret_msg = claud_client.messages.create(**request_message)
        classifications.append((index, ret_msg))
        print(classifications)
        # Update tokens used
        total_tokens_used += count_words_in_request_message(request_message)  # Assuming 100 tokens per request as per the max_tokens parameter
        if total_tokens_used >= max_tokens_per_day:
            print("Reached the daily token limit.\n")
            print(f"num or row reached = {i}\n  with the title = {dataset[i]['title']}")
            break

        # Check if the token usage is nearing the limit within the minute
        if total_tokens_used % max_tokens_per_minute == 0:
            elapsed_time = time.time() - start_time
            if elapsed_time < 60:
                time.sleep(60 - elapsed_time)
            start_time = time.time()

    return classifications

In [208]:
for i in range(5):
    print(ds['train'][i]['title']+"\n\n")
    

 ‘Maury’ Show Official Facebook Posts F*CKED UP Caption On Guest That Looks Like Ted Cruz (IMAGE)


 Trump’s Favorite News Channel Tries To Soothe His Battered Ego – Gets Taken To The Cleaners


Russia warns Iraq, Kurds not to destabilize Middle East after Kurdish vote


WATCH STEVE SCALISE Throw A Strike At The Nationals Baseball Game [Video]


 Trump Will HATE What Stephen Colbert Just Did To Him – It’s Pure Comedy Genius (VIDEO)




In [210]:
classifications = classify_articles(claud_client, ds['train'])


KeyboardInterrupt: 

In [212]:
def get_calssification_from_clade(row):    
    ret_msg = claud_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=100,
        temperature=0,
        system="Your task is to classify a given title and text of a news article as being fake news or real news."
               "return strictly a binary classification: '0' if the article is fake news, '1' if the article is real news."
               "do not return any explanations, only '0' or '1'",
        messages= Claude_transform_row_to_ZERO_shot_massage(row)
    )
    return row['index'], ret_msg

In [226]:
 ds['train'][0]['index']

0

In [ ]:
class_list = [(row['index'], get_calssification_from_clade(row)) for row in ds['train']]    